In [1]:
import random
import numpy as np
import pandas as pd
from keras.layers import Dense
from keras.models import Sequential
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
import numpy as np
import pandas_datareader.data as web
import matplotlib.pyplot as plt

Using TensorFlow backend.
C:\Users\theon\anaconda3\envs\fe690\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\theon\anaconda3\envs\fe690\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\theon\anaconda3\envs\fe690\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\theon\anaconda3\envs\fe690\lib\site-pa

In [2]:
class ENN(Sequential):
    def __init__(self, wts_sub=None):
        super().__init__()
        if wts_sub is None:
            layer1 = Dense(1, input_shape=(1,), activation='sigmoid')
            layer2 = Dense(3, activation='sigmoid')
            layer3 = Dense(3, activation='sigmoid')
            layer4 = Dense(3, activation='sigmoid')
            layer5 = Dense(1, activation='sigmoid')
            self.add(layer1)
            self.add(layer2)
            self.add(layer3)
            self.add(layer4)
            self.add(layer5)
        else:
            self.add(Dense(1,input_shape=(1,),activation='sigmoid',weights=[wts_sub[0], np.zeros(1)]))
            self.add(Dense(3,activation='sigmoid',weights=[wts_sub[1], np.zeros(3)]))
            self.add(Dense(3,activation='sigmoid',weights=[wts_sub[2], np.zeros(3)]))
            self.add(Dense(3,activation='sigmoid',weights=[wts_sub[3], np.zeros(3)]))
            self.add(Dense(1,activation='sigmoid',weights=[wts_sub[4], np.zeros(1)]))

    def f_propagation(self, X_train, y_train):
        y_predicted = self.predict(X_train.values)
        self.fitness = accuracy_score(y_train, y_predicted.round())

    def train(self, epochs):
        self.compile(optimizer='rmsprop',loss='binary_crossentropy',metrics=['accuracy'])
        self.fit(X_train.values, y_train.values, epochs=epochs)

def mutation(wts_sub):
    choose = random.randint(0, len(wts_sub)-1)
    mut = random.uniform(0, 1)
    if mut >= .5:
        wts_sub[choose] *= random.randint(2, 5)
    else:
        pass

def cross(nn1, nn2):
    nn1_weights = []
    nn2_weights = []
    wts_sub = []
    for layer in nn1.layers:
        nn1_weights.append(layer.get_weights()[0])

    for layer in nn2.layers:
        nn2_weights.append(layer.get_weights()[0])

    for i in range(0, len(nn1_weights)):
        split = random.randint(0, np.shape(nn1_weights[i])[1]-1)
        for j in range(split, np.shape(nn1_weights[i])[1]-1):
            nn1_weights[i][:, j] = nn2_weights[i][:, j]

        wts_sub.append(nn1_weights[i])

    mutation(wts_sub)

    child = ENN(wts_sub)
    return child


In [3]:
prices = pd.DataFrame()
tickers = ['^DJI']
for i in tickers:
    tmp = web.DataReader(i, 'yahoo', '1/1/2007', '01/12/2020')
    prices[i] = tmp['Adj Close']

prices['Percent Change'] = prices.pct_change()

def set_signal(column):
    if column['Percent Change'] < -0.0050:
        signal = -1
    elif column['Percent Change'] > 0.0050:
        signal = 1
    else:
        signal = 0
    return signal

prices['Liquidate'] = prices.apply(set_signal, axis=1)
prices = prices.replace(np.nan,0)
prices = prices.drop(['^DJI'], axis=1)
prices.reset_index(inplace=True,drop=True)
prices

,Percent Change,Liquidate
0,0.000000,0
1,0.000495,0
2,-0.006625,-1
3,0.002055,0
4,-0.000555,0
...,...,...
3274,0.002392,0
3275,-0.004170,0
3276,0.005647,1
3277,0.007369,1


In [4]:
X = prices['Percent Change']
X = 100*X.round(8)
X = X.astype(np.float32)
print(X.astype(np.float32))
y = prices.drop(['Percent Change'], axis=1)
#y = y.astype(np.float32)
print(y)
X_train, X_test, y_train, y_test = train_test_split(X, y)

0       0.000000
1       0.049468
2      -0.662469
3       0.205521
4      -0.055464
          ...   
3274    0.239219
3275   -0.417028
3276    0.564693
3277    0.736858
3278   -0.459755
Name: Percent Change, Length: 3279, dtype: float32
      Liquidate
0             0
1             0
2            -1
3             0
4             0
...         ...
3274          0
3275          0
3276          1
3277          1
3278          0

[3279 rows x 1 columns]


In [5]:
networks = []
pool = []
gen = 0
n = 20

for i in range(0, n):
    networks.append(ENN())

fit_max = 0

wts_opt = []

while fit_max < .9:
    gen += 1
    print('Generation', gen)

    for nn in networks:
        nn.f_propagation(X_train, y_train)
        pool.append(nn)

    networks.clear()

    pool = sorted(pool, key=lambda x: x.fitness)
    pool.reverse()

    for i in range(0, len(pool)):
        if pool[i].fitness > fit_max:
            fit_max = pool[i].fitness
            print('Fitness Score: ', fit_max)
            wts_opt = []
            for layer in pool[i].layers:
              wts_opt.append(layer.get_weights()[0])
            print(wts_opt)

    for i in range(0, 5):
        for j in range(0, 2):
            temp = cross(pool[i], random.choice(pool))
            networks.append(temp)

portfolio_enn = ENN(wts_opt)
portfolio_enn.train(10)
y_predicted = portfolio_enn.predict(X_test.values)
print('Accuracy: %.2f' % accuracy_score(y_test, y_predicted.round()))

Generation 1

Fitness Score:  0.52745018300122
[array([[-0.18007028]], dtype=float32), array([[-0.3734979, -0.6116921,  0.0135982]], dtype=float32), array([[-0.67697763, -0.3669834 , -0.98618317],
       [ 0.32843876,  0.58297896,  0.756027  ],
       [-0.29567385, -0.92154336,  0.10063314]], dtype=float32), array([[ 0.9209757 ,  0.12847257, -0.45686913],
       [ 0.15937471, -0.8362098 ,  0.2838583 ],
       [-0.02626586,  0.5696068 ,  0.58939576]], dtype=float32), array([[-0.0678612 ],
       [-0.18778503],
       [-0.43860328]], dtype=float32)]
Generation 2
Generation 3
Generation 4
Generation 5
Generation 6
Generation 7
Generation 8
Generation 9
Generation 10
Generation 11
Generation 12
Generation 13
Fitness Score:  0.7848718991459943
[array([[-1.3304343]], dtype=float32), array([[-4.1956697 , -0.74789435, -0.14174366]], dtype=float32), array([[ 0.6543691 , -0.992717  , -3.1531684 ],
       [ 0.78485894, -0.28002524,  6.1182604 ],
       [-0.8220217 , -0.24014235, -1.2660391 ]], dt

KeyboardInterrupt: 

In [ ]:
portfolio_nn = Sequential()
portfolio_nn.add(Dense(1, input_shape=(1,), activation='sigmoid'))
portfolio_nn.add(Dense(3, activation='sigmoid'))
portfolio_nn.add(Dense(3, activation='sigmoid'))
portfolio_nn.add(Dense(3, activation='sigmoid'))
portfolio_nn.add(Dense(1, activation='sigmoid'))
portfolio_nn.compile(optimizer='rmsprop',loss='hinge',metrics=['accuracy'])
portfolio_nn.fit(X_train.values, y_train.values, epochs=500)
y_predicted = portfolio_nn.predict(X_test.values)
y_predicted = np.around(y_predicted, 0)
print('Test Accuracy: %.2f' % accuracy_score(y_test, y_predicted.round()))